In [14]:
from tweepy.api import API
from tweepy.auth import AppAuthHandler, OAuthHandler
from tweepy.cursor import Cursor
from tweepy.streaming import Stream, StreamListener
import os, json
from pprint import PrettyPrinter, pformat
from TweepyCredentials import * 

In [3]:
# ACCESS_TOKEN = 'ACCESS_TOKEN'
# ACCESS_TOKEN_SECRET = 'ACCESS_TOKEN_SECRET'
# CONSUMER_KEY = 'CONSUMER_KEY'
# CONSUMER_SECRET_KEY = 'CONSUMER_SECRET_KEY'

# print(os.environ.get('T_CONSUMER_KEY'))

In [4]:
class Authenticator():
    
    def authenticate_tweets(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_KEY_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

In [50]:
class MyListener(StreamListener):
    
    def __init__(self, tweet_file):
        self.tweet_file = tweet_file
    
    def on_data(self, status):
        try: 
            with open(self.tweet_file, 'a') as tf:
                status = pformat(status, indent=4)
                tf.write(status + 'n' * 10)
        except BaseException as e:
            print('Error: ', str(e))
            pass

    def on_error(self, error):
        print(error)
        if error == 420:
            return False
        

In [52]:
if __name__ == "__main__":

    auth = Authenticator().authenticate_tweets()
    
    listener = MyListener('tweets.txt')
    
    stream = Stream(auth, listener)
    stream.filter(track = ['coca cola', 'pepsi'])


KeyboardInterrupt: 